In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver

import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from pydrake.all import StartMeshcat


In [4]:
from pydrake.geometry import (
    AddContactMaterial, Box, CollisionFilterDeclaration, Cylinder, GeometrySet,
    IllustrationProperties, MakePhongIllustrationProperties, Meshcat,
    MeshcatVisualizer, MeshcatVisualizerParams, ProximityProperties, Role,
    RoleAssign, SceneGraph, SceneGraphInspector)

In [5]:
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [6]:
builder = DiagramBuilder()
meshcat = StartMeshcat()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/RRT_cert_paper_examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.5
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,0.4], [x_fac,-1.4*sp,0.4], [-x_fac,-1.4*sp,0.4], [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()

        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star, maximum_path_degree = 1, plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity if you want
# to see the plant with the collision geometries.
# meshcat_params = MeshcatVisualizerParams()
# meshcat_params.role = Role.kIllustration
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)#, meshcat_params)
meshcat.SetProperty("/Background", "top_color", [0.8, 0.8, 0.6])
meshcat.SetProperty("/Background", "bottom_color",
                         [0.9, 0.9, 0.9])
diagram = builder.Build()

INFO:drake:Meshcat listening for connections at http://localhost:7002


Time to construct line certifier = 7.866169452667236s


In [7]:
#compute limits in s-space
limits_s = []
for q in [q_low, q_high]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)

NameError: name 'q_low' is not defined

In [ ]:
visualizer.meshcat_cspace.Delete("/prm")

In [ ]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat_cspace.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}/prm/rm/line/{plt_idx}"
#             for c in map(int, sqtring):
#                 name += f"/{c}"
            visualizer.meshcat_cspace.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle_good = partial(plot_prm, width = 0.1, prefix = "Good", color = Rgba(0, 1.,0,1))
plotting_fn_handle_bad = partial(plot_prm, width = 0.1, prefix = "Bad", color = Rgba(1., 0.,1,1))

def collision(pos, col_func_handle):
    return col_func_handle(pos)

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos)

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.check_collision_s_by_ik)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.check_collision_s_by_ik)

visualizer.check_collision_s_by_ik(np.array([0,0]))
prm_col_fn_handle(np.array([0,0]))


In [ ]:
num_points = 100

PRM = prm.PRM( 
            limits_s,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle_good
            )

PRM_bad = prm.PRM( 
            limits_s,
            num_points = num_points,
            col_func_handle = prm_col_fn_handle_bad,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle_bad
            )

# PRM.add_start_end(start, target)
if num_points < 100:
    PRM.plot()
    PRM_bad.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.show_res_q(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

In [ ]:
# This plane certifies that the two tips of the flippers don't intersect
visualizer.add_plane_indices_of_interest(118)

# re-display the sliders for convenience
for slider in sliders:
    display(slider)

In [ ]:
from pydrake.polynomial import Polynomial as PolynomialCommon

In [ ]:

def make_line_polys(PRM):
    endpoint_index_set = set()
    for neighbors in PRM.adjacency_list:
        for n in neighbors[1:]:
            endpoint_index_set.add((neighbors[0], n))
    polys = np.empty(shape=(plant.num_positions(), len(endpoint_index_set)), dtype = object)
    for i, (idx0, idx1) in enumerate(endpoint_index_set):
        s0 = PRM.nodes[idx0]
        s1 = PRM.nodes[idx1]
        for j in range(plant.num_positions()):
            polys[j,i] = PolynomialCommon(np.array([s0[j]-s1[j], s1[j]]))
    return polys

path_safe = make_line_polys(PRM)
    

In [ ]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [ ]:
t0 = time.time()
ret = cspace_free_path.FindSeparationCertificateGivenPath(path_safe, set(), cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

In [ ]:
def count_num_safe(safe_points_vect):
    num_safe = 0
    for val in safe_points_vect:
        if(val):
            num_safe += 1
    return num_safe

In [ ]:
ret[0]